<center><h1>Selección de características</h1></center>

## 1. Valores faltantes.

En el flujo de trabajo de aprendizaje automático, una vez que hemos seleccionado el modelo que queremos usar, el siguiente paso importante es seleccionar las características apropiadas para ese modelo. En esta lección, exploraremos cómo usar la correlación entre las funciones y la columna de destino, la correlación entre las funciones y la varianza de las funciones para seleccionar funciones. Continuaremos trabajando con el mismo conjunto de datos de vivienda de la última lección.

Nos centraremos específicamente en la selección de columnas de características que no tengan ningún valor faltante o que no necesiten ser transformadas para ser útiles (por ejemplo, columnas como Año de construcción y Año de remoción/agregación). Exploraremos cómo lidiar con ambos en una lección posterior de este curso.

Para comenzar, veamos qué columnas se incluyen en cualquiera de estas dos categorías.

### Ejercicio

- Lea AmesHousing.txt en un dataframe denominado `data`. Asegúrese de usar el delimitador `\t`.
- Cree un dataframe llamado `train`, que contiene las primeras 1460 filas de `data`.
- Cree un dataframe llamado `test`, que contiene el resto de las filas `data`.
- Seleccione las columnas de números enteros y flotantes de `train` y asígnelas a la variable `numerical_train`.
- Quite las siguientes columnas de `numerical_train`:
    - `PID` (la identificación del lugar no es útil para modelar)
    - `Year Built`
    - `Year Remod/Add`
    - `Garage Yr Blt`
    - `Mo Sold`
    - `Yr Sold`
- Calcule el número de valores faltantes de cada columna en `numerical_train`. Cree un objeto `pandas.Series` donde el índice se compone de nombres de columna y los valores asociados son el número de valores faltantes:
```python
Order                0
PID                  0
MS SubClass          0
MS Zoning            0
...
```
- Asigne este objeto `pandas.Series` a `null_series`. Seleccione el subconjunto de `null_series` para mantener solo las columnas sin valores faltantes y asigne el objeto `pandas.Series` resultante a `full_cols_series`.
- Muestre `full_cols_series` usando la función `print()`.

## 2. Correlación de columnas de características con la columna objetivo.

Anteriormente, seleccionamos la característica para el modelo de regresión lineal simple al comparar cómo algunas de las características se correlacionan con la columna de destino. Si recuerda, nos enfocamos en 4 características en particular y usamos el método [`pandas.DataFrame.corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) para devolver los coeficientes de correlación entre cada par de columnas. Esto significa que la matriz de correlación para 4 columnas da como resultado 16 valores de correlación:

El subconjunto de características en las que queremos centrarnos, full_cols_series, contiene 25 columnas

La matriz de correlación resultante contendrá 25 * 25 o 625 valores de correlación. Comparar y contrastar tantos valores es increíblemente difícil. En su lugar, centrémonos en cómo las variables se correlacionan con el precio (`SalePrice`).

### Ejercicio

- Calcule los coeficientes de correlación por pares entre todas las columnas en `train_subset = train[full_cols_series.index]`.

- Seleccione solo la columna SalePrice del marco de datos resultante, calcule el valor absoluto de cada término, ordene la Serie resultante por los valores de correlación y asigne a `sorted_corrs`.

## 3. Mapa de calor de matriz de correlación.
Ahora tenemos una lista decente de características candidatas para usar en nuestro modelo, ordenadas según su correlación con la columna `SalePrice`. Por ahora, mantengamos solo las características que tienen una correlación de 0.3 o superior. Este corte es un poco arbitrario y, en general, es una buena idea experimentar con este corte. Por ejemplo, puedes entrenar y probar modelos usando diferentes puntos de corte y ver dónde deja de mejorar el modelo.

Lo siguiente que debemos buscar es la posible **colinealidad** entre algunas de estas variables. La colinealidad se da cuando 2 variables (columnas) están altamente correlacionadas y corren el riesgo de duplicar información. Si tenemos 2 características que transmiten la misma información usando 2 medidas o métricas diferentes, no necesitamos conservar ambas.

Si bien podemos verificar la colinealidad entre 2 columnas usando la matriz de correlación, corremos el riesgo de sobrecarga de información. En su lugar, podemos generar un mapa de [calor de matriz de correlación](http://seaborn.pydata.org/examples/heatmap_annotation.html) usando Seaborn para comparar visualmente las correlaciones y buscar correlaciones de características por pares. Debido a que estamos buscando valores atípicos en el mapa de calor, esta representación visual es más fácil.

Así es como se ve el mapa de calor de la matriz de correlación de ejemplo de la documentación:

<img src="figs/correlation_heatmap_matrix.png" widht="800" height="600"/>


Para generar un mapa de calor de matriz de correlación, debemos pasar el dataframe que contiene la matriz de correlación como un dataframe a la función `seaborn.heatmap()`.

### Ejercicio

- Seleccione solo las columnas en `sorted_corrs` con una correlación superior a 0,3 y asígnelas a `strong_corrs`.
- Filtre `train_subset` utilizando los índices de `strong_corrs` y almacene las correlaciones en `corrmat`.
- Use la función [`seaborn.heatmap`](http://seaborn.pydata.org/generated/seaborn.heatmap.html) para generar un mapa de calor de matriz de correlación para las columnas en strong_corrs.

## 4. Modelo de entrenamiento y prueba.
Según el mapa de calor de la matriz de correlación, podemos decir que los siguientes pares de columnas están fuertemente correlacionados:

- Gr Liv Area y TotRms AbvGrd
- Garage Area y Garage Cars

Si leemos las descripciones de estas columnas de la documentación de datos, podemos decir que cada par de columnas refleja información muy similar. Debido a que `Gr Liv Area` y `Garage Area` son variables continuas que captan más matices, eliminemos `TotRms AbvGrd` y `Garage Cars`.

Lo último que debemos hacer es confirmar que el conjunto de prueba no contiene valores faltantes para estas columnas:

Parece que el conjunto de prueba tiene **una fila** en la que falta un valor para `Garage Area`. Quitemos esta fila por ahora. Finalmente, entrenemos y probemos un modelo usando estas columnas para ver cómo les va.

### Ejercicio

- Filtre el dataframe `test` para que solo contenga las columnas de `final_corr_cols.index`. Luego, quite la fila que contiene los valores faltantes y asigne el resultado a `clean_test`
- Cree un modelo de regresión lineal.
- Calcule el RMSE en los conjuntos de prueba y entrenamiento.
- Asigne el RMSE de tren a `train_rmse` y el RMSE de prueba a `test_rmse`.

## 5. Eliminación de características de baja varianza.
La última técnica que exploraremos es eliminar características con poca variación. Cuando los valores de una columna de características tienen una varianza baja, no contribuyen significativamente a la capacidad predictiva del modelo. En el extremo, imaginemos una columna con una varianza de 0. Esto significaría que todos los valores en esa columna **son exactamente iguales**. Esto significa que la columna no es informativa y no ayudará al modelo a hacer mejores predicciones.

Para hacer comparaciones de manzanas con manzanas entre columnas, necesitamos reescalar todas las columnas para que varíen entre 0 y 1. Esto se conoce como escalado mínimo-máximo (min.max) o [reescalado](https://en.wikipedia.org/wiki/Feature_scaling#Rescaling). Aquí está la fórmula para cambiar la escala:

$$\dfrac{x - \min(x)} {\max(x) - \min(x)}$$

donde

- $x$ es un valor individual
- $\min(x)$ es el valor mínimo en la columna a la que pertenece $x$
- $\max(x)$ es el valor máximo en la columna a la que pertenece $x$

### Ejercicio

- Seleccione las columnas en `features` del dataframe de entrenamiento. Vuelva a escalar cada una de las columnas para que los valores oscilen entre `0` y `1`, utilizando `train[features]` en lugar de $x$  en la fórmula anterior. Asigne el resultado a `unit_train`.
- Calcule y muestre los valores mínimo y máximo de la columna de `unit_train` para asegurarse de que todos los valores van de `0` a `1`.


## 6. Modelo Final.
Calculemos las varianzas y eliminemos las columnas con la varianza más baja.

Vemos que `Open Porch SF` es la columna con la varianza más baja y está más alejada de `Full Bath` que `Full Bath` de la siguiente.

Para concluir este ejercicio, eliminemos `Open Porch SF` y entrenemos y probemos un modelo usando las funciones restantes.

### Ejercicio
- Elimina `Open Porch SF` de las características.
- Cree un modelo de regresión lineal utilizando las funciones restantes.
- Calcule el RMSE en los conjuntos de prueba y entrenamiento.
    - Use `clean_test` 
- Asigne el RMSE de entrenamiento a `train_rmse_2` y el RMSE de prueba a `test_rmse_2`.
- Muestra ambos valores RMSE usando la función `print()`